In [53]:
import utils
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from datetime import date, timedelta
from dateutil.relativedelta import relativedelta
import datetime
from scipy.stats import norm


In [3]:
ratings,port ,yc,trans = utils.upload_data()

In [4]:
trans.head()

,AAA,AA,A,BBB,BB,B,CCC,D
AAA,0.89810,0.09346,0.00547,0.00135,0.00072,0.00050,0.00030,0.00010
AA,0.00499,0.90725,0.08081,0.00490,0.00092,0.00060,0.00031,0.00021
A,0.00031,0.01672,0.92573,0.05225,0.00301,0.00109,0.00053,0.00034
BBB,0.00019,0.00096,0.03442,0.91899,0.03783,0.00443,0.00186,0.00134
BB,0.00011,0.00033,0.00121,0.05004,0.85576,0.07476,0.01144,0.00635


In [5]:

def CDS_Price(N, coup, spread,t2m,ye,freq):
    D = (1/(1+ ye/freq)**(freq*t2m))
    result =  N *  (spread - coup )/ ye *  (1-D)
    return(result)
def beta_recovery(ratings,alpha=1.62, beta=1.86):
    num_of_companies = ratings.shape[0]
    recoveries=np.random.beta(alpha,beta, (num_of_companies, 1))
    ratings["Recovery"] = recoveries
    return(ratings)

ratings_w_recov = beta_recovery(ratings)

In [6]:

port["Unique_id"] = np.arange(len(port))
port.head()

,Instrument type (Bond or CDS),CUSIP,Notional,Maturity Date,Coupon,Coupons per year,Price,Yield,Theoretical Clean Price,Expected Recovery rate,Unique_id
Name,,,,,,,,,,,
BELL CANADA,Bond,07813ZAL8,7,2032-04-01,0.078500,2,145.161,0.027605,145.186022,40,0
Atlantic Power LP,Bond,04878TAA5,7,2037-06-23,0.059500,2,91.683,0.068285,91.682925,40,1
Atlantic Power LP,CDS,0,5,2027-01-01,0.044015,2,0.000,0.024270,0.000000,60,2
Rogers Communications Inc,Bond,775109AP6,7,2040-11-03,0.066800,2,146.504,0.033350,146.533443,40,3
Riocan Real Estate Investment,Bond,766910AW3,7,2024-04-17,0.037250,2,104.860,0.015597,104.859058,40,4


In [7]:
def yc_spread(yc):
    # print(yc.loc[:, yc.columns != 'Government'])
    #print(yc.loc[:,yc.columns != "Government"])
    Wo_Gov =  yc.loc[:, yc.columns != "Government"]
    Wo_Gov_1 = Wo_Gov.loc[:, Wo_Gov.columns != "Tenor"]
    output = Wo_Gov_1 - np.array(yc.loc[:, yc.columns == "Government"])
    return(output)

In [8]:
ratings.set_index("Name",inplace=True)

In [9]:
yc_spreads = yc_spread(yc)
yc_spreads.head()

,AAA,AA,A,BBB,BB,B,CCC
In years,,,,,,,
0.25,0.000750,0.000900,0.001250,0.003550,0.010360,0.011070,0.024635
0.50,0.000270,0.000400,0.000740,0.003020,0.010190,0.011050,0.024820
1.00,0.000620,0.000710,0.001020,0.003250,0.011100,0.012290,0.026525
2.00,0.000480,0.000590,0.000920,0.003300,0.012930,0.014760,0.030255
3.00,0.000515,0.000625,0.001015,0.003675,0.014735,0.016995,0.033625


In [10]:
def Correlation_Matrix(ratings,port):
    """
    Correlation_Matrix consume  a pd.DataFrame of portfolio positions
    and returns the associated correaltion matrix for each instrument

    Remark: 

    At the  moment correaltion matrix is filled on the diagnonal with 1s and
    0.36 The code will be changed at a later date
    """
    Comp_ = pd.unique(port.index)
    #print(Comp_)
    n_assets = len(Comp_) 
    
    Cor_Matrix = np.diag([1 for i in range(n_assets)])
    
    Cor_matrix_pd =  pd.DataFrame(Cor_Matrix, index = Comp_, columns=Comp_)
    # Slow Method 
    for comp1 in Cor_matrix_pd.index:
        correlation = 0
        for comp2 in Cor_matrix_pd.columns:
            if comp1 == comp2:
                correlation = 1 
            elif ratings.loc[comp1, "Industry"] == ratings.loc[comp2, "Industry"]:
                correlation = 0.65
            else: 
                correlation = 0.28
            Cor_matrix_pd.loc[comp1,comp2] = correlation
    
    
    return(Cor_matrix_pd)


In [11]:
covar= Correlation_Matrix(ratings,port)

In [12]:
port.head()

,Instrument type (Bond or CDS),CUSIP,Notional,Maturity Date,Coupon,Coupons per year,Price,Yield,Theoretical Clean Price,Expected Recovery rate,Unique_id
Name,,,,,,,,,,,
BELL CANADA,Bond,07813ZAL8,7,2032-04-01,0.078500,2,145.161,0.027605,145.186022,40,0
Atlantic Power LP,Bond,04878TAA5,7,2037-06-23,0.059500,2,91.683,0.068285,91.682925,40,1
Atlantic Power LP,CDS,0,5,2027-01-01,0.044015,2,0.000,0.024270,0.000000,60,2
Rogers Communications Inc,Bond,775109AP6,7,2040-11-03,0.066800,2,146.504,0.033350,146.533443,40,3
Riocan Real Estate Investment,Bond,766910AW3,7,2024-04-17,0.037250,2,104.860,0.015597,104.859058,40,4


In [13]:
port_rating = pd.merge(port, ratings_w_recov, on="Name")

In [14]:
port_rating

,Instrument type (Bond or CDS),CUSIP,Notional,Maturity Date,Coupon,Coupons per year,Price,Yield,Theoretical Clean Price,Expected Recovery rate,Unique_id,Issuer Rating,Industry,Recovery
Name,,,,,,,,,,,,,,
BELL CANADA,Bond,07813ZAL8,7,2032-04-01,0.078500,2,145.161,0.027605,145.186022,40,0,BBB,Telecom,0.187404
Atlantic Power LP,Bond,04878TAA5,7,2037-06-23,0.059500,2,91.683,0.068285,91.682925,40,1,BB,Energy,0.675475
Atlantic Power LP,CDS,0,5,2027-01-01,0.044015,2,0.000,0.024270,0.000000,60,2,BB,Energy,0.675475
Atlantic Power LP,Bond,04878TAA5,7,2037-06-23,0.059500,2,90.694,0.069401,90.693722,40,16,BB,Energy,0.675475
Atlantic Power LP,CDS,0,5,2027-01-01,0.045131,2,0.000,0.024270,0.000000,60,17,BB,Energy,0.675475
Atlantic Power LP,Bond,04878TAA5,7,2037-06-23,0.059500,2,90.694,0.069401,90.693722,40,37,BB,Energy,0.675475
Atlantic Power LP,CDS,0,5,2027-01-01,0.045131,2,0.000,0.024270,0.000000,60,38,BB,Energy,0.675475
Rogers Communications Inc,Bond,775109AP6,7,2040-11-03,0.066800,2,146.504,0.033350,146.533443,40,3,BBB,Telecom,0.187638
Rogers Communications Inc,Bond,775109AP6,7,2040-11-03,0.066800,2,145.696,0.033804,145.725011,40,15,BBB,Telecom,0.187638


In [15]:
def time_maturity(portfolio,start_date = date(2021,1,1), time_lead=0):
    """
        time_maturity consumes a pd.DataFrame representing the portfolio and a datetime object called 'start_date'
        and returns a portfolio pd.DataFrame with 'T2M' column representing the "Time to Maturity"

        time_maturity: pd.DataFrame, datetime.date -> pd.DataFrame
    """

    portfolio["T2M"] = portfolio["Maturity Date"].apply(lambda x: (relativedelta(x,start_date).years +(relativedelta(x,start_date).months/12))) - time_lead
    portfolio["T2M"] = np.where(portfolio["T2M"] <= 0, portfolio["T2M"] + 1, portfolio["T2M"]) # checks for negative dates,  and adds a year.
    return(portfolio)

In [16]:
def adjust_yield_curve(port,yc, time_lead=0):
    col_names = yc.columns 
    col_names.drop(["Government"])
    ret = port.copy()
    time_now = date(2021,1,1)
    #ret["status"]= ret["Instrument type (Bond or CDS)"].apply(lambda x: 1 if x == "Bond" else 0 )
    # ret["Years_Remaining"] =  ret["Maturity_Date"].apply(lambda x: (relativedelta(x,now_w).years +(relativedelta(x,now_w).months/12))) - 1
    # ret["Years_remaining"] = np.wherse(ret["Years Remaining"] <= 0, ret["Years Remaining"] + 1, ret["Years Remaining"])
    ret = time_maturity(ret, time_now, time_lead) 
    
    #result = pd.DataFrame(columns=col_names)
    
    ### Adjusts yield _cur
    df =  yc.copy()
    for i in ret["T2M"]:
        if i not in yc.index:
            df.loc[i] = np.nan
   
    df.sort_index(axis= 0,inplace = True)
    df = df.interpolate(method="linear",axis=0) #new yc with times in between time to maturities.
    df.fillna(axis= 0, method="bfill", inplace = True) 
    # back fills rates for short term rates
    df = df[df.index.notnull()] # removes na indexes
    return(df)

In [17]:
### Part 2 of PART 1 code here
def market_val_port(port):
    l = port.Price * port.Notional /100
    l.columns = ["Name","Portfolio Value"]
    return(l)

def routine_price(portfolio,yc):
    # Set index in years to help make it easier for interpolation
    yc = yc.set_index("In years")
    """
    Description:
    Routine_price simulates the values of each bond and CDS relative to change in credit rating returning
    the simulated prices based on a transition matrix, and yield associated with the rating.

    Purpose:
    Using this procedure will reflect the price of the financial instrument based on credit rather for each credit rating.
    after a 1 year time step
    Variates:
    Routine_price: 
    - objtype: pd.DataFrame 
    - dtype: pd.float64
    - Matrix containing prices for each instrument.
    - Columns: The credit rating
    - rows: The  instrument and its prices at every Rating after passing 1 year
    
    portfolio: 
    - objtype: pd.DataFrame
    - dtype: pd.float64
    - Matrix containing portfolio of financial instruments and their associated information.
    
    yc:
    - objtype: pd.DataFrame
    - dtype: pd.float64
    - Matrix containing yields of bonds with respect to Credit ratings.
    """
    ### Todays date
    now_w = datetime.datetime.now()
    ## TODO Create routine such that for defaults result in Recovery value for Bonds, and Notional * recovery %
    # def makehams_formula(coup, rate, time_remaining, freq, Notional):
    #     ye = math.floor(time_remaining)
    #     adj = time_remaining - ye
    #     ## TODO adjust adj  such that we only need to consider the period after coupon.
        
    #     if adj > 0.5:
    #         time_to_maturity = ye + 0.5 
    #         adj  = (adj- 0.5)/0.5
    #     else:
    #         time_to_maturity = ye 
    #         adj = adj / 0.5
    #     # D is the discounting factor
    #     D = (1/(1+ rate/freq)**(freq* time_to_maturity))
    #     # P is the Clean price of the bond at the specific 
    #     Dirty = coup/rate * Notional *(1 - D)  + Notional * D
    #     # Dirty_price represents the Dirty Price of the bond
    #     P = Dirty * (1 + rate/2)**adj  -   Notional*coup*adj
    #     return(P, Dirty) 




    def makehams_formula(coup, rate, time_remaining, freq, Notional):
        ye = np.floor(time_remaining) 
        adj = time_remaining - ye
        t2 = ye.copy()
        print(adj)
        ## TODO adjust adj  such that we only need to consider the period after coupon.
        def check_adj(adj):
            if adj > 0.5:
                #time_to_maturity = ye + 0.5 
                adj  = (adj- 0.5)/0.5
            else:
                
                adj = adj / 0.5
            return(adj)
        def check_adj_2(adj):
            if adj > 0.5:
                #time_to_maturity = ye + 0.5 
                k =  0.5
            else:
                k = 0
            return(k)
        t1 = adj.copy()
        adjust = adj.apply(lambda x: check_adj(x)) 
        time_to_maturity = ye + t1.apply(lambda x: check_adj_2(x))

        # D is the discounting factor
        D = (1/(1+ rate/freq)**(freq*time_to_maturity))
        # P is the Clean price of the bond at the specific 
        Dirty = (coup/rate * Notional *(1 - D)  + Notional * D)
        # Dirty_price represents the Dirty Price of the bond
        P = Dirty * (1 + rate/2)**adjust  -   Notional*coup*adjust
        return(pd.from_dict({"Clean Price":P,"Dirty: Price" : Dirty },dtype= np.float64)) 
    
    # time_1 = now_w + timedelta(365.25)
    def CDS_pricer(status,Instrument,recovery,Notional):
        if Instrument == "CDS":
            if  status == "Default":
                return(Notional * recovery)
            else:
                return(0)
    
    
    

#df.loc[df['set_of_numbers'] <= 4, 'equal_or_lower_than_4?'] = 'True' 
#df.loc[df['set_of_numbers'] > 4, 'equal_or_lower_than_4?'] = 'False'
            

    def run_pricer(port):
    # TODO create a condition for CDSs and one for Bonds
    # I've created a column denoting CDS's
        #port.loc[df["Instrument type (Bond or CDS)"] == "Bond", 'Instrument Price'] 
        #port = price_CDS(port)
        #port = price_bonds(port)
        #return(port)
        pass

    
    # def one_step(port, rating):
    col_names = yc.columns
    col_names.remove("government","In years")
    ret = portfolio.copy()

    ret["status"]= ret["Instrument type (Bond or CDS)"].apply(lambda x: 1 if x == "Bond" else 0 )
    ret["Years_Remaining"] =  ret["Maturity_Date"].apply(lambda x: (relativedelta(x,now_w).years +(relativedelta(x,now_w).months/12))) - 1
    ret["Years_remaining"] = np.wherse(ret["Years Remaining"] <= 0, ret["Years Remaining"] + 1, ret["Years Remaining"])
    result = pd.DataFrame(columns=col_names)
    
    df =  yc.copy()
    for i in ret["Years_Remaining"]:
        if i not in yc.index:
            df.loc[i] = np.nan
   
    df.sort_index(axis= 0,inplace = True)
    df = df.interpolate(method="linear",axis=0) #new yc with times in between time to maturities.
    df.fillna(axis= 0, method="bfill", inplace = True) 
    # back fills rates for short term rates
    df = df[df.index.notnull()] # removes na indexes
    
    
    # Interporlate each rate. linear
    
    #ret["Instrument"] = ret["Instrument type (Bond or CDS)"].apply(lambda x: 1 if x  == "Bond")
    #TODO check if "In years" contains else interperolate

    pass

In [18]:
temp_yc = adjust_yield_curve(port,yc)
temp_yc_1year_fws = adjust_yield_curve(port,yc,time_lead=1)

In [19]:
temp_yc.head()

,Tenor,Government,AAA,AA,A,BBB,BB,B,CCC
In years,,,,,,,,,
0.250000,,0.00040,0.001150,0.001300,0.00165,0.00395,0.010760,0.011470,0.025035
0.500000,6M,0.00060,0.000870,0.001000,0.00134,0.00362,0.010790,0.011650,0.025420
1.000000,1Y,0.00070,0.001320,0.001410,0.00172,0.00395,0.011800,0.012990,0.027225
1.250000,2Y,0.00078,0.001372,0.001466,0.00178,0.00404,0.012246,0.013564,0.028051
1.416667,2Y,0.00086,0.001424,0.001522,0.00184,0.00413,0.012692,0.014138,0.028877


In [20]:
port2 = time_maturity(port_rating)
port2.head()

,Instrument type (Bond or CDS),CUSIP,Notional,Maturity Date,Coupon,Coupons per year,Price,Yield,Theoretical Clean Price,Expected Recovery rate,Unique_id,Issuer Rating,Industry,Recovery,T2M
Name,,,,,,,,,,,,,,,
BELL CANADA,Bond,07813ZAL8,7,2032-04-01,0.078500,2,145.161,0.027605,145.186022,40,0,BBB,Telecom,0.187404,11.250000
Atlantic Power LP,Bond,04878TAA5,7,2037-06-23,0.059500,2,91.683,0.068285,91.682925,40,1,BB,Energy,0.675475,16.416667
Atlantic Power LP,CDS,0,5,2027-01-01,0.044015,2,0.000,0.024270,0.000000,60,2,BB,Energy,0.675475,6.000000
Atlantic Power LP,Bond,04878TAA5,7,2037-06-23,0.059500,2,90.694,0.069401,90.693722,40,16,BB,Energy,0.675475,16.416667
Atlantic Power LP,CDS,0,5,2027-01-01,0.045131,2,0.000,0.024270,0.000000,60,17,BB,Energy,0.675475,6.000000


In [21]:
temp2 = port2.copy()

In [22]:
temp2["Grade rate"] = temp_yc.lookup(temp2["T2M"], temp2["Issuer Rating"])


/Users/rickyyung/anaconda3/envs/generative/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The 'lookup' method is deprecated and will beremoved in a future version.You can use DataFrame.melt and DataFrame.locas a substitute.
  """Entry point for launching an IPython kernel.


In [23]:
temp2.head()

,Instrument type (Bond or CDS),CUSIP,Notional,Maturity Date,Coupon,Coupons per year,Price,Yield,Theoretical Clean Price,Expected Recovery rate,Unique_id,Issuer Rating,Industry,Recovery,T2M,Grade rate
Name,,,,,,,,,,,,,,,,
BELL CANADA,Bond,07813ZAL8,7,2032-04-01,0.078500,2,145.161,0.027605,145.186022,40,0,BBB,Telecom,0.187404,11.250000,0.024755
Atlantic Power LP,Bond,04878TAA5,7,2037-06-23,0.059500,2,91.683,0.068285,91.682925,40,1,BB,Energy,0.675475,16.416667,0.052206
Atlantic Power LP,CDS,0,5,2027-01-01,0.044015,2,0.000,0.024270,0.000000,60,2,BB,Energy,0.675475,6.000000,0.027982
Atlantic Power LP,Bond,04878TAA5,7,2037-06-23,0.059500,2,90.694,0.069401,90.693722,40,16,BB,Energy,0.675475,16.416667,0.052206
Atlantic Power LP,CDS,0,5,2027-01-01,0.045131,2,0.000,0.024270,0.000000,60,17,BB,Energy,0.675475,6.000000,0.027982


In [24]:
temp2["ISpread"] = temp2["Yield"] - temp2["Grade rate"]

In [25]:
temp2[(temp2["ISpread"] < 0) & (temp2["Instrument type (Bond or CDS)"]  == "Bond")]

,Instrument type (Bond or CDS),CUSIP,Notional,Maturity Date,Coupon,Coupons per year,Price,Yield,Theoretical Clean Price,Expected Recovery rate,Unique_id,Issuer Rating,Industry,Recovery,T2M,Grade rate,ISpread
Name,,,,,,,,,,,,,,,,,
Rogers Communications Inc,Bond,775109AP6,7,2040-11-03,0.06680,2,146.504,0.033350,146.533443,40,3,BBB,Telecom,0.187638,19.833333,0.034149,-0.000798
Rogers Communications Inc,Bond,775109AP6,7,2040-11-03,0.06680,2,145.696,0.033804,145.725011,40,15,BBB,Telecom,0.187638,19.833333,0.034149,-0.000345
Rogers Communications Inc,Bond,775109AP6,7,2040-11-03,0.06680,2,145.696,0.033804,145.725011,40,36,BBB,Telecom,0.187638,19.833333,0.034149,-0.000345
SOBEYS INC,Bond,83357ZAD9,7,2036-10-28,0.06060,2,122.495,0.040292,122.510633,40,7,BB,Food,0.776990,15.750000,0.050241,-0.009949
INTER PIPELINE LTD,Bond,45834ZAA7,7,2022-08-02,0.04967,2,102.838,0.001215,102.825767,40,21,BBB,Energy,0.401880,1.583333,0.004220,-0.003005
MORGAN STANLEY,Bond,61746BDT0,7,2022-08-05,0.03125,2,101.697,0.002658,101.689560,40,22,BBB,Banking,0.105944,1.583333,0.004220,-0.001562


In [26]:
ratings.head()
number_of_companies = ratings.shape[0]
print(number_of_companies)

22


In [27]:
def beta_recovery(ratings,alpha=1.62, beta=1.86):
    num_of_companies = ratings.shape[0]
    recoveries=np.random.beta(alpha,beta, (num_of_companies, 1))
    ratings["Recovery"] = recoveries
    return(ratings)
    
    

In [28]:
def calculate_recovery(recov, Notional):
    return(recov* Notional)

In [29]:
def CDS_Price(N, coup, spread,t2m,ye,freq):
    D = (1/(1+ ye/freq)**(freq*t2m))
    result =  N *  (spread - coup )/ ye *  (1-D)
    return(result)


In [30]:
# I want to split the data to two different portfolios 

def one_year_forward(port,transition,yc):
    # Comment you want to use the yc for one year later: We will get new years 
    comp_uniq = np.unique(port.index)
    grades = np.unique(transition.columns)
    result = pd.DataFrame(index = comp_uniq,columns= grades)
    temp = port.copy()
    #TODO LOOKUP Table and return rates add the idiosyncratic adjustment and bam.
    # First we must reduce  the T2M by one year
    temp["T2M_1yr"] = temp["T2M"] - 1 
    # If maturities after 1 year are less than 0 set to 1
    temp["T2M_1yr"] = np.where(temp["T2M_1yr"] <0 , 1 , temp["T2M_1yr"])
    #temp2["Grade rate"] = temp.lookup(temp2["T2M"], temp2["Issuer Rating"]
        # temp_grades  = grades.tolist()
    # temp_grades.remove("D") # remove default from considerations
    desired_col = ["Unique_id", "Instrument type (Bond or CDS)", "T2M_1yr","Coupon","ISpread", "Coupons per year",
     "Notional","Recovery", "Yield"]
    
    grades_list = grades.tolist()
    #grades_list.remove("D")

    result = temp[desired_col]
    result.reset_index(inplace=True)
    temp_v  = pd.DataFrame({"New Ratings": grades_list})
    result_f = pd.merge(result, temp_v, how="cross")
    result_f.set_index("Name", inplace=True)
    yc["D"] = 0.5
    result_f["Sim Yields"] = yc.lookup(result_f["T2M_1yr"], result_f["New Ratings"])
    result_f["New Yields"]  = (result_f["ISpread"].values + result_f["Sim Yields"].T).T
    
    
    result_f["Sim Prices"] = np.where(result_f["Instrument type (Bond or CDS)"] == "Bond", utils.makehams_formula(
        result_f.Coupon, result_f["New Yields"], result_f["T2M_1yr"],result_f["Coupons per year"],result_f["Notional"]
    ), CDS_Price(result_f["Notional"],result_f["Coupon"],result_f["New Yields"],result_f["T2M_1yr"],result_f["New Yields"],result_f["Coupons per year"]))
    Step3 = result_f.copy()
    Step3["Sim Prices"] = np.where((Step3["New Ratings"] == "D") & (Step3["Instrument type (Bond or CDS)"] == "Bond"), calculate_recovery(Step3["Recovery"], Step3["Notional"]), Step3["Sim Prices"])
    Step3["Sim Prices"] =  np.where((Step3["New Ratings"] == "D") & (Step3["Instrument type (Bond or CDS)"] == "CDS"), calculate_recovery(1-Step3["Recovery"], Step3["Notional"]),Step3["Sim Prices"])


    #Step3[(Step3["New Ratings"] == "D") & (Step3["Instrument type (Bond or CDS)"] == "Bond")]["Sim Prices"] = calculate_recovery(Step3["Recovery"], Step3["Notional"] )    
    #Step3[(Step3["New Ratings"] == "D") & (Step3["Instrument type (Bond or CDS)"] == "Bond")]["Sim Prices"] = calculate_recovery((1-Step3["Recovery"]), Step3["Notional"] )
    
    return(Step3)

In [31]:
temp2.head()

,Instrument type (Bond or CDS),CUSIP,Notional,Maturity Date,Coupon,Coupons per year,Price,Yield,Theoretical Clean Price,Expected Recovery rate,Unique_id,Issuer Rating,Industry,Recovery,T2M,Grade rate,ISpread
Name,,,,,,,,,,,,,,,,,
BELL CANADA,Bond,07813ZAL8,7,2032-04-01,0.078500,2,145.161,0.027605,145.186022,40,0,BBB,Telecom,0.187404,11.250000,0.024755,0.002849
Atlantic Power LP,Bond,04878TAA5,7,2037-06-23,0.059500,2,91.683,0.068285,91.682925,40,1,BB,Energy,0.675475,16.416667,0.052206,0.016080
Atlantic Power LP,CDS,0,5,2027-01-01,0.044015,2,0.000,0.024270,0.000000,60,2,BB,Energy,0.675475,6.000000,0.027982,-0.003712
Atlantic Power LP,Bond,04878TAA5,7,2037-06-23,0.059500,2,90.694,0.069401,90.693722,40,16,BB,Energy,0.675475,16.416667,0.052206,0.017196
Atlantic Power LP,CDS,0,5,2027-01-01,0.045131,2,0.000,0.024270,0.000000,60,17,BB,Energy,0.675475,6.000000,0.027982,-0.003712


In [32]:
yc_1yr = adjust_yield_curve(temp2, yc,1)

In [33]:
scenarios = one_year_forward(temp2 ,trans,yc_1yr)
scenarios.head()

/Users/rickyyung/anaconda3/envs/generative/lib/python3.7/site-packages/ipykernel_launcher.py:29: FutureWarning: The 'lookup' method is deprecated and will beremoved in a future version.You can use DataFrame.melt and DataFrame.locas a substitute.


,Unique_id,Instrument type (Bond or CDS),T2M_1yr,Coupon,ISpread,Coupons per year,Notional,Recovery,Yield,New Ratings,Sim Yields,New Yields,Sim Prices
Name,,,,,,,,,,,,,
BELL CANADA,0,Bond,10.25,0.0785,0.002849,2,7,0.187404,0.027605,A,0.019698,0.022547,10.273252
BELL CANADA,0,Bond,10.25,0.0785,0.002849,2,7,0.187404,0.027605,AA,0.018163,0.021012,10.393318
BELL CANADA,0,Bond,10.25,0.0785,0.002849,2,7,0.187404,0.027605,AAA,0.017548,0.020397,10.441900
BELL CANADA,0,Bond,10.25,0.0785,0.002849,2,7,0.187404,0.027605,B,0.045610,0.048460,8.480481
BELL CANADA,0,Bond,10.25,0.0785,0.002849,2,7,0.187404,0.027605,BB,0.041690,0.044540,8.725467


In [34]:
test = scenarios.copy()
test["Sim Prices"] = np.where((test["New Ratings"] == "D") & (test["Instrument type (Bond or CDS)"] == "Bond"), calculate_recovery(test["Recovery"], test["Notional"]), test["Sim Prices"])
test["Sim Prices"] =  np.where((test["New Ratings"] == "D") & (test["Instrument type (Bond or CDS)"] == "CDS"), calculate_recovery(1-test["Recovery"], test["Notional"]),test["Sim Prices"])


In [35]:
test[test["New Ratings"] == "D"].head()

,Unique_id,Instrument type (Bond or CDS),T2M_1yr,Coupon,ISpread,Coupons per year,Notional,Recovery,Yield,New Ratings,Sim Yields,New Yields,Sim Prices
Name,,,,,,,,,,,,,
BELL CANADA,0,Bond,10.250000,0.078500,0.002849,2,7,0.187404,0.027605,D,0.5,0.502849,1.311829
Atlantic Power LP,1,Bond,15.416667,0.059500,0.016080,2,7,0.675475,0.068285,D,0.5,0.516080,4.728322
Atlantic Power LP,2,CDS,5.000000,0.044015,-0.003712,2,5,0.675475,0.024270,D,0.5,0.496287,1.622627
Atlantic Power LP,16,Bond,15.416667,0.059500,0.017196,2,7,0.675475,0.069401,D,0.5,0.517196,4.728322
Atlantic Power LP,17,CDS,5.000000,0.045131,-0.003712,2,5,0.675475,0.024270,D,0.5,0.496287,1.622627


In [36]:
trans.astype("float64")

,AAA,AA,A,BBB,BB,B,CCC,D
AAA,0.89810,0.09346,0.00547,0.00135,0.00072,0.00050,0.00030,0.00010
AA,0.00499,0.90725,0.08081,0.00490,0.00092,0.00060,0.00031,0.00021
A,0.00031,0.01672,0.92573,0.05225,0.00301,0.00109,0.00053,0.00034
BBB,0.00019,0.00096,0.03442,0.91899,0.03783,0.00443,0.00186,0.00134
BB,0.00011,0.00033,0.00121,0.05004,0.85576,0.07476,0.01144,0.00635
B,0.00009,0.00023,0.00078,0.00166,0.05152,0.85105,0.05657,0.03810
CCC,0.00008,0.00030,0.00107,0.00177,0.00649,0.14714,0.50897,0.33417
D,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,1.00000


In [37]:
trans[trans.columns[::-1]]

,D,CCC,B,BB,BBB,A,AA,AAA
AAA,0.00010,0.00030,0.00050,0.00072,0.00135,0.00547,0.09346,0.89810
AA,0.00021,0.00031,0.00060,0.00092,0.00490,0.08081,0.90725,0.00499
A,0.00034,0.00053,0.00109,0.00301,0.05225,0.92573,0.01672,0.00031
BBB,0.00134,0.00186,0.00443,0.03783,0.91899,0.03442,0.00096,0.00019
BB,0.00635,0.01144,0.07476,0.85576,0.05004,0.00121,0.00033,0.00011
B,0.03810,0.05657,0.85105,0.05152,0.00166,0.00078,0.00023,0.00009
CCC,0.33417,0.50897,0.14714,0.00649,0.00177,0.00107,0.00030,0.00008
D,1.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000


# Simulating results

In [57]:
def ratings_sim(port,trans,covar,ratings,  N=2000):
    #portfolio_w_ratings=portfolio_w_ratings.reset_index()
    #portfolio_w_ratings = pd.merge(port, ratings, left_on=["Name"], right_on=["Name"])
    #TODO take a normal random draw with Cholesky L  then multiply and get new numbers and convert to letters
    #results = np.zeros(port.shape[0],N)
    n_comp = len(port.index.unique())
    normal_draws= np.random.standard_normal((n_comp, N))
    L =  np.linalg.cholesky(covar)
    correlated_draws = norm.cdf(L.T @  normal_draws)
    
    # TODO Now that we have correlated draws we now have to turn the 
    t_bounds =  trans[trans.columns[::-1]].cumsum(axis=1 )

    Step4  = pd.DataFrame(correlated_draws,index=port.index.unique() )    
    
    Step4 = Step4.join(ratings)
    def look_for_rating(x, cdftrans, initial_rating):
        d= cdftrans.loc[[initial_rating]]
        look = d.T
        d=look[look[initial_rating].le(x)].index[-1]
        return(d)         

    # building table of ratings only for companies
    for  Scenario in range(N):
        for name in Step4.index:
                try:
                    Step4.loc[name,Scenario] = look_for_rating(Step4.loc[name,Scenario], t_bounds, ratings.loc[name,"Issuer Rating"])
                except:
                    Step4.loc[name,Scenario] = "D"
    Step4["Instrument type (Bond or CDS)"] = port["Instrument type (Bond or CDS)"]   
    
    #Step4 = Step4.set_index(["Name","Instrument type (Bond or CDS)"])
    
    return(Step4)

In [45]:
def look_for_rating(x, cdftrans, initial_rating):
    d= cdftrans.loc[[initial_rating]]
    look = d.T
    e=look[look[initial_rating].le(x)].index[-1]
    return(e)    

In [58]:
s= ratings_sim(temp2,trans,covar,ratings,  N=2000)
s.head()

IndexError: index -1 is out of bounds for axis 0 with size 0

In [50]:
look_for_rating(0.2,temp_cdftrans, ratings.loc["BELL CANADA","Issuer Rating"])

'BB'

In [ ]:
trans.loc[["AAA"]]

,AAA,AA,A,BBB,BB,B,CCC,D
AAA,0.8981,0.09346,0.00547,0.00135,0.00072,0.0005,0.0003,0.0001


In [54]:
n_comp = len(temp2.index.unique())
normal_draws= np.random.standard_normal((n_comp, 10))
L =  np.linalg.cholesky(covar)
# correlated_draws = norm.cdf(L.T @  normal_draws)

In [56]:
correlated_draws =  L.T @ normal_draws
e = norm.cdf(correlated_draws)
Step4  = pd.DataFrame(e,index=temp2.index.unique() )
Step4.loc["BELL CANADA", 1]

0.9872662790945033

In [41]:
temp2.head()

,Instrument type (Bond or CDS),CUSIP,Notional,Maturity Date,Coupon,Coupons per year,Price,Yield,Theoretical Clean Price,Expected Recovery rate,Unique_id,Issuer Rating,Industry,Recovery,T2M,Grade rate,ISpread
Name,,,,,,,,,,,,,,,,,
BELL CANADA,Bond,07813ZAL8,7,2032-04-01,0.078500,2,145.161,0.027605,145.186022,40,0,BBB,Telecom,0.187404,11.250000,0.024755,0.002849
Atlantic Power LP,Bond,04878TAA5,7,2037-06-23,0.059500,2,91.683,0.068285,91.682925,40,1,BB,Energy,0.675475,16.416667,0.052206,0.016080
Atlantic Power LP,CDS,0,5,2027-01-01,0.044015,2,0.000,0.024270,0.000000,60,2,BB,Energy,0.675475,6.000000,0.027982,-0.003712
Atlantic Power LP,Bond,04878TAA5,7,2037-06-23,0.059500,2,90.694,0.069401,90.693722,40,16,BB,Energy,0.675475,16.416667,0.052206,0.017196
Atlantic Power LP,CDS,0,5,2027-01-01,0.045131,2,0.000,0.024270,0.000000,60,17,BB,Energy,0.675475,6.000000,0.027982,-0.003712


In [ ]:
Step4.head()

NameError: name 'Step4' is not defined

In [ ]:
result = temp2.copy()
result.reset_index(inplace=True)
temp_v  = pd.DataFrame({"Grades": trans.index})
#temp_v.reset_index(inplace=True)
result_f = pd.merge(result, temp_v, how="cross")
result_f.head()
    

,Name,Instrument type (Bond or CDS),CUSIP,Notional,Maturity Date,Coupon,Coupons per year,Price,Yield,Theoretical Clean Price,Expected Recovery rate,Unique_id,Issuer Rating,Industry,Recovery,T2M,Grade rate,ISpread,Grades
0,BELL CANADA,Bond,07813ZAL8,7,2032-04-01,0.0785,2,145.161,0.027605,145.186022,40,0,BBB,Telecom,0.395179,11.25,0.024755,0.002849,AAA
1,BELL CANADA,Bond,07813ZAL8,7,2032-04-01,0.0785,2,145.161,0.027605,145.186022,40,0,BBB,Telecom,0.395179,11.25,0.024755,0.002849,AA
2,BELL CANADA,Bond,07813ZAL8,7,2032-04-01,0.0785,2,145.161,0.027605,145.186022,40,0,BBB,Telecom,0.395179,11.25,0.024755,0.002849,A
3,BELL CANADA,Bond,07813ZAL8,7,2032-04-01,0.0785,2,145.161,0.027605,145.186022,40,0,BBB,Telecom,0.395179,11.25,0.024755,0.002849,BBB
4,BELL CANADA,Bond,07813ZAL8,7,2032-04-01,0.0785,2,145.161,0.027605,145.186022,40,0,BBB,Telecom,0.395179,11.25,0.024755,0.002849,BB


In [42]:
temp_cdftrans = trans[trans.columns[::-1]].cumsum(axis=1)
temp_cdftrans

,D,CCC,B,BB,BBB,A,AA,AAA
AAA,0.00010,0.00040,0.00090,0.00162,0.00297,0.00844,0.10190,1.00000
AA,0.00021,0.00052,0.00112,0.00204,0.00694,0.08775,0.99500,0.99999
A,0.00034,0.00087,0.00196,0.00497,0.05722,0.98295,0.99967,0.99998
BBB,0.00134,0.00320,0.00763,0.04546,0.96445,0.99887,0.99983,1.00002
BB,0.00635,0.01779,0.09255,0.94831,0.99835,0.99956,0.99989,1.00000
B,0.03810,0.09467,0.94572,0.99724,0.99890,0.99968,0.99991,1.00000
CCC,0.33417,0.84314,0.99028,0.99677,0.99854,0.99961,0.99991,0.99999
D,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000


In [ ]:
def look_for_rating(x, cdftrans, initial_rating):
    d= cdftrans.loc[[initial_rating]]
    look = d.T
    rating=look[look[initial_rating].le(x)].index[-1]
    return(rating)

In [ ]:
look_for_rating(0.2,temp_cdftrans,"B")

'CCC'